# **Классификация видов риса**

Датасет состоит из 18185 строк, каждая строка содержит данные о отдельном зерне риса. Представлен рис двух видов: Jasmine (Class = 1) и Gonen (Class = 0).

Даны такие параметры зерен как:

* Площадь (Area)
* Длина главной оси (MajorAxisLength)
* Длина побочной оси (MinorAxisLength)
* Эксцентриситет (Eccentricity)
* Выпуклая площадь (ConvexArea)
* Эквивалентный диаметр (EquivDiameter)
* Экстент (Площадь проекции) (Extent)
* Периметр (Perimeter)
* Округлость (Roundness)
* Соотношение сторон (AspectRation)

Это набор данных  содержит классификацию риса. 
Необходимо предсказать, к какому из двух классов риса принадлежит рис:

In [37]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 
import seaborn as sns           
import sklearn
import plotly.express          as ex
from sklearn.model_selection import GridSearchCV

In [2]:
filename = 'riceClassification.csv'

# Загрузка датасета
data = pd.read_csv(filename)
classes = ['Gonen', 'Jasmine']

In [3]:
# Размеры таблицы
data.shape

Посмотрим первые строки таблицы

In [4]:
data.head()

In [5]:
data.info()

Из приведенных выше выходных данных ясно, что в наборе данных нет категориальных столбцов.

In [6]:
num_cols = [x for x in data.columns if data[x].dtype in ['int64', 'float64'] ]

# **Визуализация данных**

Получаем сводную информацию о признаках: 

In [7]:
data.describe()

In [8]:
d = data.copy()
d["Class"] =  d["Class"].map({1:'Class : 1',0:'Class : 0'})
ex.pie(d,names='Class',title='Rice Classes')

Как видно, классы более менее сбалансированны.

Теперь построим матрицу корреляции 

In [9]:
corr_mat = data.corr()
corr_mat

In [10]:
plt.figure(figsize=(20,20))
sns.heatmap(data=pd.get_dummies(data).corr(), annot=True);

Мы видим, что MinorAxisLength,AspectRation,Roundness,Area,ConvexArea,EquivDiameter и Eccentricity имеют очень высокую корреляцию с целевой переменной.

Построим диаграммы рассеивания для данных признаков:

In [11]:
sns.pairplot(data=data, hue='Class');
pass

По графикам можно заметить что почти для каждого из измерений тренды для классов различаются. Это значит, что даже по нескольким измерениям можно достаточно надёжно предсказать класс. Также можно заметить однозначную зависимость эксцентриситета и соотношения сторон - эти величины зависимы или зависят от одного и того же набора других величин.

In [12]:
fig, axes = plt.subplots(5,2, figsize=(20,25))
fig.suptitle('Box Plot Before Outlier Detection')
k = 0
for i in range(0,5):
    for j in range(0,2):
        sns.boxplot(ax=axes[i,j], data=data, x=data.columns[k], palette='Dark2')
        k=k+1

Из графиков выше мы видим, что в пяти параметрах у нас бльшое количество выбросов.

In [13]:
# Вычисляем IQR
def iqr_calculation(main_df,col):
    Q1 = np.percentile(col,25,interpolation = 'midpoint')
    Q3 = np.percentile(col,75,interpolation = 'midpoint')
    IQR = Q3 - Q1
    upper = np.where(col>=(Q3+1.5*IQR))
    lower = np.where(col<=(Q1-1.5*IQR))
    return upper,lower
    
# Убираем выбросы
def remove_outliers(main_df,upper,lower):
    main_df.drop(upper[0], inplace = True)
    main_df.drop(lower[0], inplace = True)
    print("New Shape: ", main_df.shape)
    return main_df

# Сбрасываем индекс фрейма данных
def reset_dataframe(main_df):
    main_df.reset_index(inplace=True)
    main_df.drop(columns=['index'],inplace=True)
    return main_df

In [14]:
main_data = data

for i in range(0,len(main_data.columns)-1):
    print('For ',data.columns[i])
    upper, lower = iqr_calculation(main_data,data[data.columns[i]])
    main_data = remove_outliers(main_data,upper,lower)
    final_data = reset_dataframe(main_data)

In [15]:
fig, axes = plt.subplots(5,2, figsize=(20,25))
fig.suptitle('Box Plot After Removing the Outliers')
k = 0
for i in range(0,5):
    for j in range(0,2):
        col_name = final_data.columns[k]
        sns.boxplot(ax=axes[i,j], data=final_data,x=col_name,palette='Dark2')
        k=k+1

# **Пропущенные значения**

In [16]:
data.isna().sum()

In [17]:
data_num = data[num_cols]
data_num.describe()

Выполним стандартизацию (линейное преобразование, приводящее все значения к нулевому среднему и единичному стандартному отклонению) всех количественных признаков:

In [18]:
data_num = (data_num - data_num.mean(axis = 0))/data_num.std(axis = 0)
data_num.describe()

# **Разделение данных на обучающую и тестовую выборки**

In [19]:
X = data
y = data['Class']
N, d = X.shape
N, d

In [20]:
X.columns

In [21]:
y

Разобьем данные на обучающую и тестовую выборки

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 85)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 

print(N_train, N_test)

# **Классификатор ближайших соседей $k$NN**

In [35]:
from sklearn.neighbors import KNeighborsClassifier

knn = GridSearchCV(KNeighborsClassifier(), param_grid = {'n_neighbors': range(1, 20)}, cv = 5)
knn.fit(X_train, y_train)

err_test = 1 - knn.best_score_

In [24]:
err_test

In [25]:
knn.predict(X_test)[0:5]

In [26]:
# Точность модели
knn.score(X_test, y_test)

In [27]:
knn.best_params_

 Оптимальным значением для n_neighbors является 11

In [29]:
# Ошибки на обучающей выборке
np.mean(y_train != knn.predict(X_train))

In [30]:
# Ошибки на тестовой выборке
np.mean(y_test  != knn.predict(X_test))

In [31]:
#SVC
from sklearn.svm import SVC
svc = SVC(gamma = 'auto')
svc.fit(X_train, y_train)

err_train = np.mean(y_train != svc.predict(X_train))
err_test  = np.mean(y_test  != svc.predict(X_test))

print(err_train, err_test)

In [ ]:
#GBT
gbt = ensemble.GradientBoostingClassifier(n_estimators = 100)
gbt.fit(X_train, y_train)

y_train_pred = gbt.predict(X_train)
y_test_pred = gbt.predict(X_test)

np.mean(y_train != y_train_pred), np.mean(y_test != y_test_pred)

# **Выводы**

*  По результатам оказалось что выборка более менее сбалансированная
*  Ни по результатам SVC,ни по результатм GBT ошибок не наблюдается, значит что оба алгоритма нам подходят 
